In [1]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import pandas as pd
import matplotlib.pyplot as plt
# import numpy.linalg as LA
# import tensorflow as tf
# import torch

from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/My Drive/Bot!')
%cd drive/My\ Drive/Bot!/

Mounted at /content/drive
/content/drive/My Drive/Bot!


In [2]:
# path = 'MarchtoMay_TweetsandNews_fortopicmodeling.csv'
# path = 'Processed.csv'
path = 'balanced.csv'
data = pd.read_csv(path)
df = data[['Content','SourceatBot7']]
df = df[df['SourceatBot7'].str.contains("Human") | df['SourceatBot7'].str.contains("Bot")]
df

,Content,SourceatBot7
0,face covid-19 success medit natur 's real heal...,Bot
1,rt murkymoodr idiot would expect shop tj maxx ...,Bot
2,rt joegood governor sign execut order 202 ny 3...,Bot
3,potus never sleep `` know '' openamerica openo...,Bot
4,realdonaldtrump brianstelt trump terribl coron...,Bot
...,...,...
53417,limeylizzi doc vaselin neosporin nostril catch...,Human
53418,rt paulkrugman actual peopl ’ dead bodi liter ...,Human
53419,yalabast ’ kill grandpar plus two week ’ go fi...,Human
53420,coronavirus worst global crisi sinc world war ...,Human


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(df['Content'])
vector = vectorizer.transform(df['Content'])

# debug
print(vector.shape)
print(type(vector))
mydict = vectorizer.vocabulary_
print(len(mydict))
print(dict(list(mydict.items())[0:2]))

(53422, 89065)
<class 'scipy.sparse.csr.csr_matrix'>
89065
{'face': 36175, 'covid': 27740}


In [4]:
X = vector
# X = np.asarray(vector)
y = 1*(df['SourceatBot7'] == 'Bot').to_numpy() # Bot = 1, Human = 0
# print(X[0])
# print(X.shape)
print(y)
print(y.shape)
print(np.count_nonzero(y == 0))
print(np.count_nonzero(y == 1))
print('ratio of human:', np.count_nonzero(y == 0)/y.shape[0])
print('ratio of bot:', np.count_nonzero(y == 1)/y.shape[0])

[1 1 1 ... 0 0 0]
(53422,)
26711
26711
ratio of human: 0.5
ratio of bot: 0.5


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [6]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
import sklearn.metrics as metrics

def classifier(clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print('='*80)
  print(clf)
  print('='*80)
  acc = accuracy_score(y_pred, y_test)
  print('Acc:', format(acc,".4f"))
  print('CM:\n', confusion_matrix(y_test, y_pred))
  tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
  recall = tp / (tp+fn)
  precision = tp / (tp+fp)
  print('recall:\n',recall)
  print("precision:\n", precision)
  f1 = 2*(precision*recall)/(precision+recall)
  print("f1:\n", f1)
  print("f1):\n", f1_score(y_test, y_pred))
  # fpr, tpr, threshold = metrics.roc_curve(y_test, y_pred)
  # roc_auc = metrics.roc_auc_score(y_test, y_pred)
  # print('Area under curve (AUC) = ',roc_auc)
  # plt.xlabel('False Positive Rate')
  # plt.ylabel('True Positive Rate')
  # plt.title('ROC Curve')
  # plt.plot(fpr,tpr)
  acc_list.append(acc)
  f1_list.append(f1)
  return acc

In [7]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.linear_model import RidgeClassifier, Perceptron, PassiveAggressiveClassifier, SGDClassifier, LogisticRegression
from sklearn.svm import LinearSVC

acc_list = []
f1_list = []

classifier(LogisticRegression())
classifier(BernoulliNB())
# classifier(MultinomialNB())
classifier(LinearSVC())

# classifier(MultinomialNB(alpha=.01))
# classifier(KNeighborsClassifier(n_neighbors=2))
# classifier(NearestCentroid())
# classifier(RidgeClassifier())
# classifier(Perceptron())
# classifier(PassiveAggressiveClassifier())
# classifier(SGDClassifier())
print(acc_list)
print(f1_list)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()
Acc: 0.6464
CM:
 [[3494 1787]
 [1991 3413]]
recall:
 0.6315692079940785
precision:
 0.6563461538461538
f1:
 0.6437193511882308
f1):
 0.6437193511882308
BernoulliNB()
Acc: 0.6645
CM:
 [[3655 1626]
 [1959 3445]]
recall:
 0.6374907475943745
precision:
 0.6793531847761782
f1:
 0.6577565632458234
f1):
 0.6577565632458234
LinearSVC()
Acc: 0.6362
CM:
 [[3361 1920]
 [1967 3437]]
recall:
 0.6360103626943006
precision:
 0.6415904424117976
f1:
 0.6387882167084843
f1):
 0.6387882167084843
[0.6464202152550305, 0.6644829199812822, 0.6362189985961628]
[0.6437193511882308, 0.6577565632458234, 0.6387882167084843]


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
